In [1]:
from pyspark.sql import SparkSession, SQLContext
from pyspark import SparkContext, SparkConf
import os
import json

In [2]:
DATA_PATH = '../data/French/'
files = os.listdir(DATA_PATH)
files_path = [DATA_PATH+f for f in files]
conf = SparkConf().setAppName("question1").setMaster("local[*]")
sc = SparkContext(conf=conf)
spark = SparkSession(sc)
tweets = spark.read.json(files_path)

In [3]:
lines = sc.textFile(','.join(files_path))

In [4]:
lines = lines.map(lambda line: json.loads(line))

In [5]:
import datetime
date_time_begin_str = 'Sat Feb 1 20:00:00 +0000 2020'
date_time_end_str = 'Sun Feb 2 20:00:00 +0000 2020'
date_time_pattern = '%a %b %d %H:%M:%S %z %Y'
date_time_begin_obj = datetime.datetime.strptime(date_time_begin_str, date_time_pattern)
date_time_end_obj = datetime.datetime.strptime(date_time_end_str, date_time_pattern)

In [23]:
def filter_time_range(s):
    print(s.keys())
    curr_datetime = datetime.datetime.strptime(s['created_at'],date_time_pattern)
    return curr_datetime > date_time_begin_obj and curr_datetime < date_time_end_obj
a = lines.filter(filter_time_range)\
    .map(lambda d: d['text'])\
    .flatMap(lambda s: s.strip().split())

In [24]:
b = a.map(lambda x: (x, 1))\
    .reduceByKey(lambda a, b: a + b)\
    .max(key=lambda x: x[1])

In [25]:
b

('RT', 48409)